<a href="https://colab.research.google.com/github/sirius70/NLP_HW2/blob/main/shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interpreting Classifier Weights

In this experiment, you will train models to distringuish examples of two different genres of Shakespeare's plays: comedies and tragedies. (We'll ignore the histories, sonnets, etc.) Since he died four hundred years ago, Shakespeare has not written any more plays—although scraps of various other works have come to light. We are not, therefore, interested in building models simply to help categorize an unbounded stream of future documents, as we might be in other applications of text classification; rather, we are interested in what a classifier might have to tell us about what we mean by the terms “comedy” and “tragedy”.

You will start by copying and running your `createBasicFeatures` function from the experiment with movie reviews. Do the features the classifier focuses on tell you much about comedy and tragedy in general?

You will then implement another featurization function `createInterestingFeatures`, which will focus on only those features you think are informative for distinguishing between comedy and tragedy. Accuracy on leave-one-out cross-validation may go up, but it more important to look at the features given the highest weight by the classifier. Interpretability in machine learning, of course, may be harder to define than accuracy—although accuracy at some tasks is hard enoough.

In [1]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_extraction.text import TfidfVectorizer
import re

In [2]:
#read in the shakespeare corpus
def readShakespeare():
  raw = requests.get("https://raw.githubusercontent.com/dasmiq/cs6120-assignment2/refs/heads/main/shakespeare_plays.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  #remove histories from the data, as we're only working with tragedies and comedies
  corpus = [entry for entry in corpus if entry["genre"] != "history"]
  return corpus

This is where you will implement two functions to featurize the data:

In [47]:
# TODO: Implement createBasicFeatures
# NB: The current contents are for testing only
# This function should return:
#  -a sparse numpy matrix of document features
#  -a list of the correct genre for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix.

# This function should create a feature representation using all tokens that
# contain an alphabetic character.

def createBasicFeatures(corpus):
    # Extract texts and genres
    texts = [entry["text"] for entry in corpus]
    genres = [entry["genre"] for entry in corpus]

    # Create vectorizer that uses all tokens containing alphabetic characters
    vectorizer = CountVectorizer(
        token_pattern=r'[a-zA-Z]+'  # This pattern matches tokens with at least one alphabetic character
    )

    # Transform texts to feature matrix
    X = vectorizer.fit_transform(texts)

    # Get vocabulary
    vocab = vectorizer.get_feature_names_out().tolist()

    print(vocab)
    print(len(vocab))

    # Return the numeric matrix, labels, and vocabulary
    return X, genres, vocab

In [15]:
# TODO: Implement createInterestingFeatures. Describe your features and what
# they might tell you about the difference between comedy and tragedy.
# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.

def createInterestingFeatures(corpus):
  #Your code here
  texts = [entry["text"] for entry in corpus]
  genres = [entry["genre"] for entry in corpus]

  vectorizer = TfidfVectorizer(
      token_pattern=r'[a-zA-Z]+',  # alphabetic tokens only
        ngram_range=(1, 2),          # unigrams + bigrams
        min_df=3,                    # ignore very rare words/phrases
        max_df=0.8,                  # drop terms that appear in >80% of plays
        sublinear_tf=True            # dampens effect of very frequent terms
  )

  X = vectorizer.fit_transform(texts)
  vocab = vectorizer.get_feature_names_out().tolist()
  print(len(vocab))

  return X, genres, vocab


In [5]:
print(createBasicFeatures(readShakespeare()))

['a', 'aaron', 'abandon', 'abandoned', 'abash', 'abate', 'abated', 'abatement', 'abatements', 'abates', 'abbess', 'abbey', 'abbominable', 'abbreviated', 'abed', 'abetting', 'abhominable', 'abhor', 'abhorr', 'abhorred', 'abhorring', 'abhors', 'abhorson', 'abide', 'abides', 'abilities', 'ability', 'abject', 'abjectly', 'abjects', 'abjur', 'abjure', 'able', 'abler', 'aboard', 'abode', 'abominable', 'abominably', 'abominations', 'abortive', 'abound', 'about', 'above', 'abr', 'abraham', 'abram', 'abreast', 'abridg', 'abridge', 'abridged', 'abridgment', 'abroach', 'abroad', 'abrogate', 'abruption', 'abruptly', 'absence', 'absent', 'absolute', 'absolutely', 'absolv', 'absolver', 'abstemious', 'abstinence', 'abstract', 'absurd', 'abundance', 'abundant', 'abundantly', 'abus', 'abuse', 'abused', 'abuser', 'abuses', 'abusing', 'aby', 'abysm', 'academe', 'academes', 'accent', 'accents', 'accept', 'acceptance', 'accepted', 'accepts', 'access', 'accessary', 'accessible', 'accidence', 'accident', 'ac

In [37]:
print(createInterestingFeatures(readShakespeare()))

274887
(<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 516819 stored elements and shape (26, 274887)>, ['comedy', 'tragedy', 'comedy', 'comedy', 'tragedy', 'tragedy', 'tragedy', 'tragedy', 'tragedy', 'comedy', 'tragedy', 'comedy', 'comedy', 'comedy', 'comedy', 'comedy', 'tragedy', 'tragedy', 'comedy', 'comedy', 'tragedy', 'tragedy', 'tragedy', 'comedy', 'comedy', 'comedy'], array(['a', 'a a', 'a an', ..., 'zounds you', 'zwagger', 'zwagger d'],
      dtype=object))


In [51]:
from sklearn.feature_extraction.text import CountVectorizer

"""
    Creates bigram features for classification.

    Features:
      - Only bigrams (sequences of two consecutive words).
      - Captures short phrases that may signal genre differences.
        For example:
            - Comedy might have bigrams like "funny joke", "haha laugh".
            - Tragedy might have bigrams like "death scene", "dark fate".

    Returns:
      - X: sparse matrix of bigram features
      - y: list of correct genres (labels)
      - vocab: list of bigram features in the vocabulary
  """

def createInterestingFeatures(corpus):
  texts = [entry["text"] for entry in corpus]
  genres = [entry["genre"] for entry in corpus]

  vectorizer = CountVectorizer(
      ngram_range=(2, 2),     # only bigrams
      token_pattern=r'[a-zA-Z]+'  # keep tokens with alphabetic chars
  )

  X = vectorizer.fit_transform(texts)
  vocab = vectorizer.get_feature_names_out()

  print(len(vocab))

  return X, genres, vocab


In [52]:
#given a numpy matrix representation of the features for the training set, the
# vector of true classes for each example, and the vocabulary as described
# above, this computes the accuracy of the model using leave one out cross
# validation and reports the most indicative features for each class
def evaluateModel(X,y,vocab,penalty="l1"):
  #create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=LeaveOneOut())

  #determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)

  #determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  neg_class_prob_sorted = model.coef_[0, :].argsort()
  pos_class_prob_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  pos_indicators = [vocab[i] for i in neg_class_prob_sorted[:termsToTake]]
  neg_indicators = [vocab[i] for i in pos_class_prob_sorted[:termsToTake]]

  return avg_score,pos_indicators,neg_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)


In [8]:
corpus = readShakespeare()

Run the following to train and evaluate two models with basic features:

In [9]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

['a', 'aaron', 'abandon', 'abandoned', 'abash', 'abate', 'abated', 'abatement', 'abatements', 'abates', 'abbess', 'abbey', 'abbominable', 'abbreviated', 'abed', 'abetting', 'abhominable', 'abhor', 'abhorr', 'abhorred', 'abhorring', 'abhors', 'abhorson', 'abide', 'abides', 'abilities', 'ability', 'abject', 'abjectly', 'abjects', 'abjur', 'abjure', 'able', 'abler', 'aboard', 'abode', 'abominable', 'abominably', 'abominations', 'abortive', 'abound', 'about', 'above', 'abr', 'abraham', 'abram', 'abreast', 'abridg', 'abridge', 'abridged', 'abridgment', 'abroach', 'abroad', 'abrogate', 'abruption', 'abruptly', 'absence', 'absent', 'absolute', 'absolutely', 'absolv', 'absolver', 'abstemious', 'abstinence', 'abstract', 'absurd', 'abundance', 'abundant', 'abundantly', 'abus', 'abuse', 'abused', 'abuser', 'abuses', 'abusing', 'aby', 'abysm', 'academe', 'academes', 'accent', 'accents', 'accept', 'acceptance', 'accepted', 'accepts', 'access', 'accessary', 'accessible', 'accidence', 'accident', 'ac

Run the following to train and evaluate two models with features that are interesting for distinguishing comedy and tragedy:

In [53]:
X,y,vocab = createInterestingFeatures(corpus)
runEvaluation(X, y, vocab)

255076
----------L1 Norm-----------
The model's average accuracy is 0.692308
The most informative terms for pos are: ['not a', 'and i', 'me and', 'which is', 'the duke', 'if you', 'you have', 'she is', 'to me', 'my love', 'the king', 'i am', 'sir toby', 'of syracuse', 'it is', 'quirk sir', 'quintus if', 'quintus martius', 'quit thee', 'quit their']
The most informative terms for neg are: ['the gods', 'art thou', 'the moor', 'give me', 'lady macbeth', 'my lord', 'to the', 'in his', 'know not', 'exeunt scene', 'caesar s', 'quirks and', 'quintus martius', 'quit thee', 'quit their', 'quit them', 'quit this', 'quit thy', 'quit with', 'quit you']
----------L2 Norm-----------
The model's average accuracy is 0.807692
The most informative terms for pos are: ['the duke', 'sir toby', 'i am', 'if you', 'and i', 'princess of', 'i pray', 'of france', 'sir i', 'she is', 'of syracuse', 'a good', 'in love', 'of a', 'to her', 'i will', 'the king', 'my love', 'antipholus of', 'you will']
The most informa

**TODO**: Based on the most informative features in the output of the classifier evaluation, what do these classifiers tell you about the differences between comedy and tragedy?